In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from collections import Counter

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score, classification_report, confusion_matrix

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import log_loss

# Dataset-1

In [2]:
from sklearn.datasets import load_breast_cancer

In [3]:
data1 = load_breast_cancer()
data1['data']

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [4]:
len(data1['data'][0])

30

In [5]:
X = data1.data#.info()
X = pd.DataFrame(X)
X

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [6]:
y = data1.target

In [7]:
def stratified_split_data(X, y, test_size):
    return train_test_split(X, y, test_size=test_size, stratify=y, random_state=42)

In [8]:
splits = {
        "20/80": stratified_split_data(X, y, test_size=0.8),
        "50/50": stratified_split_data(X, y, test_size=0.5),
        "80/20": stratified_split_data(X, y, test_size=0.2),
    }

## Random Forest

In [9]:
rf = RandomForestClassifier()

In [10]:
params =  {
        "n_estimators": [50, 100, 200],
        "max_depth": [None, 10, 20]
    }

In [11]:
def evaluate_classifier(clf, params, X_train, X_test, y_train, y_test):
    # Perform grid search with cross-validation
    grid_search = GridSearchCV(clf, param_grid=params, cv=StratifiedKFold(3), scoring="accuracy", return_train_score=True)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    
    # Training error
    y_train_pred = best_model.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_error = 1 - train_accuracy

    # Validation error (mean across folds)
    validation_error = 1 - grid_search.best_score_

    # Testing error
    y_test_pred = best_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_error = 1 - test_accuracy

    # Print metrics
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Training Accuracy: {train_accuracy:.4f} (Error: {train_error:.4f})")
    print(f"Validation Accuracy: {grid_search.best_score_:.4f} (Error: {validation_error:.4f})")
    print(f"Testing Accuracy: {test_accuracy:.4f} (Error: {test_error:.4f})")
    print(classification_report(y_test, y_test_pred))
    print(f"Confusion Matrix:\n{confusion_matrix(y_test, y_test_pred)}")

    return {
        "train_error": train_error,
        "validation_error": validation_error,
        "test_error": test_error,
        "best_params": grid_search.best_params_
    }

### Getting Accuracy reports for our different training/test split for Random Forest

In [12]:
for partition, (X_train, X_test, y_train, y_test) in splits.items():
        print(f"\n--- Results for {partition} Split ---")
        result = evaluate_classifier(rf, params, X_train, X_test, y_train, y_test)
        print(result)
        #results[dataset_name][partition][clf_name] = result


--- Results for 20/80 Split ---
Best parameters: {'max_depth': None, 'n_estimators': 200}
Training Accuracy: 1.0000 (Error: 0.0000)
Validation Accuracy: 0.9644 (Error: 0.0356)
Testing Accuracy: 0.9364 (Error: 0.0636)
              precision    recall  f1-score   support

           0       0.98      0.85      0.91       170
           1       0.92      0.99      0.95       286

    accuracy                           0.94       456
   macro avg       0.95      0.92      0.93       456
weighted avg       0.94      0.94      0.94       456

Confusion Matrix:
[[144  26]
 [  3 283]]
{'train_error': 0.0, 'validation_error': np.float64(0.03556187766714081), 'test_error': 0.0635964912280702, 'best_params': {'max_depth': None, 'n_estimators': 200}}

--- Results for 50/50 Split ---
Best parameters: {'max_depth': None, 'n_estimators': 50}
Training Accuracy: 1.0000 (Error: 0.0000)
Validation Accuracy: 0.9682 (Error: 0.0318)
Testing Accuracy: 0.9474 (Error: 0.0526)
              precision    recal

## Decision Tree

In [13]:
params_dt = {
        "max_depth": [None, 10, 20],
        "min_samples_split": [2, 5, 10]
}

In [14]:
dt = DecisionTreeClassifier()

### Getting Accuracy reports for our different training/test split for Decision Tree

In [15]:
for partition, (X_train, X_test, y_train, y_test) in splits.items():
    print(f"\n--- Results for {partition} Split ---")
    result = evaluate_classifier(dt, params_dt, X_train, X_test, y_train, y_test)
    print(result)


--- Results for 20/80 Split ---
Best parameters: {'max_depth': None, 'min_samples_split': 2}
Training Accuracy: 1.0000 (Error: 0.0000)
Validation Accuracy: 0.9471 (Error: 0.0529)
Testing Accuracy: 0.8794 (Error: 0.1206)
              precision    recall  f1-score   support

           0       0.90      0.76      0.82       170
           1       0.87      0.95      0.91       286

    accuracy                           0.88       456
   macro avg       0.89      0.85      0.87       456
weighted avg       0.88      0.88      0.88       456

Confusion Matrix:
[[129  41]
 [ 14 272]]
{'train_error': 0.0, 'validation_error': np.float64(0.05286865813181618), 'test_error': 0.12061403508771928, 'best_params': {'max_depth': None, 'min_samples_split': 2}}

--- Results for 50/50 Split ---
Best parameters: {'max_depth': 20, 'min_samples_split': 2}
Training Accuracy: 1.0000 (Error: 0.0000)
Validation Accuracy: 0.9471 (Error: 0.0529)
Testing Accuracy: 0.9123 (Error: 0.0877)
              precision

## KNN

In [16]:
params_knn = {
        "n_neighbors": [3, 5, 10],
        "weights": ["uniform", "distance"]
    }

In [17]:
knn = KNeighborsClassifier()

### Getting Accuracy reports for our different training/test split for KNN

In [18]:
for partition, (X_train, X_test, y_train, y_test) in splits.items():
        print(f"\n--- Results for {partition} Split ---")
        result = evaluate_classifier(knn, params_knn, X_train, X_test, y_train, y_test)
        print(result)


--- Results for 20/80 Split ---
Best parameters: {'n_neighbors': 3, 'weights': 'uniform'}
Training Accuracy: 0.9115 (Error: 0.0885)
Validation Accuracy: 0.9203 (Error: 0.0797)
Testing Accuracy: 0.9101 (Error: 0.0899)
              precision    recall  f1-score   support

           0       0.96      0.79      0.87       170
           1       0.89      0.98      0.93       286

    accuracy                           0.91       456
   macro avg       0.92      0.89      0.90       456
weighted avg       0.91      0.91      0.91       456

Confusion Matrix:
[[135  35]
 [  6 280]]
{'train_error': 0.08849557522123896, 'validation_error': np.float64(0.07965860597439545), 'test_error': 0.08991228070175439, 'best_params': {'n_neighbors': 3, 'weights': 'uniform'}}

--- Results for 50/50 Split ---
Best parameters: {'n_neighbors': 5, 'weights': 'distance'}
Training Accuracy: 1.0000 (Error: 0.0000)
Validation Accuracy: 0.9367 (Error: 0.0633)
Testing Accuracy: 0.9333 (Error: 0.0667)
             

## Naive Bayes

### Getting Accuracy reports for our different training/test split for Naive Bayes

In [19]:
# Define parameter grid for GaussianNB
naive_bayes_params = {
    "var_smoothing": [1e-9, 1e-8, 1e-7]  # Smoothing parameter for GaussianNB
}

# Run evaluation
for partition, (X_train, X_test, y_train, y_test) in splits.items():
    print(f"\n--- Results for {partition} Split ---")
    result = evaluate_classifier(GaussianNB(), naive_bayes_params, X_train, X_test, y_train, y_test)
    print(result)


--- Results for 20/80 Split ---
Best parameters: {'var_smoothing': 1e-07}
Training Accuracy: 0.9558 (Error: 0.0442)
Validation Accuracy: 0.9467 (Error: 0.0533)
Testing Accuracy: 0.9167 (Error: 0.0833)
              precision    recall  f1-score   support

           0       0.99      0.79      0.88       170
           1       0.89      0.99      0.94       286

    accuracy                           0.92       456
   macro avg       0.94      0.89      0.91       456
weighted avg       0.92      0.92      0.91       456

Confusion Matrix:
[[134  36]
 [  2 284]]
{'train_error': 0.04424778761061943, 'validation_error': np.float64(0.053342816500711265), 'test_error': 0.08333333333333337, 'best_params': {'var_smoothing': 1e-07}}

--- Results for 50/50 Split ---
Best parameters: {'var_smoothing': 1e-09}
Training Accuracy: 0.9401 (Error: 0.0599)
Validation Accuracy: 0.9365 (Error: 0.0635)
Testing Accuracy: 0.9439 (Error: 0.0561)
              precision    recall  f1-score   support

      

## SVM

### Getting Accuracy reports for our different training/test split for SVM

In [20]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import warnings

# Define a pipeline with scaling and SVM
svm_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", SVC(probability=True))  # Set probability=True to enable ROC-AUC computation
])

# Define parameter grid
svm_params = {
    "svm__C": [0.1, 1, 10],        # Regularization strength
    "svm__kernel": ["linear", "rbf"],
    "svm__gamma": ["scale", "auto"]  # Kernel coefficient for rbf
}

# Run evaluation
for partition, (X_train, X_test, y_train, y_test) in splits.items():
    print(f"\n--- Results for {partition} Split ---")
    result = evaluate_classifier(svm_pipeline, svm_params, X_train, X_test, y_train, y_test)
    print(result)



--- Results for 20/80 Split ---
Best parameters: {'svm__C': 0.1, 'svm__gamma': 'scale', 'svm__kernel': 'linear'}
Training Accuracy: 0.9912 (Error: 0.0088)
Validation Accuracy: 0.9822 (Error: 0.0178)
Testing Accuracy: 0.9605 (Error: 0.0395)
              precision    recall  f1-score   support

           0       0.99      0.91      0.94       170
           1       0.95      0.99      0.97       286

    accuracy                           0.96       456
   macro avg       0.97      0.95      0.96       456
weighted avg       0.96      0.96      0.96       456

Confusion Matrix:
[[154  16]
 [  2 284]]
{'train_error': 0.008849557522123908, 'validation_error': np.float64(0.01778093883357046), 'test_error': 0.03947368421052633, 'best_params': {'svm__C': 0.1, 'svm__gamma': 'scale', 'svm__kernel': 'linear'}}

--- Results for 50/50 Split ---
Best parameters: {'svm__C': 1, 'svm__gamma': 'scale', 'svm__kernel': 'linear'}
Training Accuracy: 0.9930 (Error: 0.0070)
Validation Accuracy: 0.9789 (Er

## Logistic Regression

### Getting Accuracy reports for our different training/test split for Logistic Regression

In [21]:
logistic_reg_params = {"C": [0.1, 1, 10], "max_iter": [7000, 10000]}

for partition, (X_train, X_test, y_train, y_test) in splits.items():
    print(f"\n--- Results for {partition} Split ---")
    result = evaluate_classifier(LogisticRegression(), logistic_reg_params, X_train, X_test, y_train, y_test)
    print(result)


--- Results for 20/80 Split ---
Best parameters: {'C': 0.1, 'max_iter': 7000}
Training Accuracy: 0.9646 (Error: 0.0354)
Validation Accuracy: 0.9559 (Error: 0.0441)
Testing Accuracy: 0.9320 (Error: 0.0680)
              precision    recall  f1-score   support

           0       0.97      0.85      0.90       170
           1       0.92      0.98      0.95       286

    accuracy                           0.93       456
   macro avg       0.94      0.91      0.93       456
weighted avg       0.93      0.93      0.93       456

Confusion Matrix:
[[144  26]
 [  5 281]]
{'train_error': 0.03539823008849563, 'validation_error': np.float64(0.04409672830725453), 'test_error': 0.06798245614035092, 'best_params': {'C': 0.1, 'max_iter': 7000}}

--- Results for 50/50 Split ---
Best parameters: {'C': 10, 'max_iter': 7000}
Training Accuracy: 0.9859 (Error: 0.0141)
Validation Accuracy: 0.9648 (Error: 0.0352)
Testing Accuracy: 0.9579 (Error: 0.0421)
              precision    recall  f1-score   suppo

# Dataset 2

In [22]:
#from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
#adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
#X = adult.data.features 
#y = adult.data.targets 
  
# metadata 
#print(adult.metadata) 
  
# variable information 
#adult.variables
#X.head()

In [23]:
data2 = pd.read_csv('heart_disease_uci.csv')
#data2.dropna(inplace=True)
data2.head()

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


In [24]:
X = data2.drop(columns = ['num', 'id'])
X.head()

,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal
0,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect
1,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal
2,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect
3,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal
4,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal


In [27]:
# Identify categorical columns
categorical_columns = [
    'sex', 'dataset', 'cp',
    'fbs', 'restecg','exang', 'slope','thal'
]
numerical_columns = [
    'age','trestbps', 'chol', 
    'thalch', 'oldpeak', 'ca'
    
]

In [28]:
def handle_missing_values(X):
    # Fill missing values in numerical columns with the mean
    X[numerical_columns] = X[numerical_columns].fillna(X[numerical_columns].mean())

    # Fill missing values in categorical columns with the mode
    X[categorical_columns] = X[categorical_columns].fillna(X[categorical_columns].mode().iloc[0])
    
    return X

In [29]:
X = handle_missing_values(X)

C:\Users\rkhus\AppData\Local\Temp\ipykernel_2164\2827648026.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[categorical_columns] = X[categorical_columns].fillna(X[categorical_columns].mode().iloc[0])


In [30]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


# Create the preprocessing transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_columns),
         ('num', StandardScaler(), numerical_columns)
    ])

def preprocess_data(X):
    # Fit and transform the entire dataset
    X_transformed = preprocessor.fit_transform(X)

    # Get feature names for numerical and categorical columns
    num_feature_names = numerical_columns
    cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_columns)

    # Combine feature names
    feature_names = list(num_feature_names) + list(cat_feature_names)

    # Create a DataFrame from the transformed array
    X_transformed_df = pd.DataFrame(
        X_transformed, 
        columns=feature_names, 
        index=X.index  # Preserve the original index
    )

    return X_transformed_df

# Optional: If you want to keep numerical columns as well
def preprocess_data_full(X):
    # Fit and transform the data
    X_transformed = preprocessor.fit_transform(X)

    # Get feature names for numerical and categorical columns
    num_feature_names = numerical_columns
    cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_columns)

    # Combine feature names
    feature_names = list(num_feature_names) + list(cat_feature_names)

    # Create a DataFrame from the transformed array
    X_transformed_df = pd.DataFrame(
        X_transformed, 
        columns=feature_names, 
        index=X.index  # Preserve the original index
    )

    return X_transformed_df

In [31]:
X = preprocess_data_full(X)
X

,age,trestbps,chol,thalch,oldpeak,ca,sex_Female,sex_Male,dataset_Cleveland,dataset_Hungary,...,restecg_normal,restecg_st-t abnormality,exang_False,exang_True,slope_downsloping,slope_flat,slope_upsloping,thal_fixed defect,thal_normal,thal_reversable defect
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.007386,0.698041,0.311021,0.495698,1.349421,-1.249371e+00
1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.432034,1.511761,0.797713,-1.175955,0.589832,4.292099e+00
2,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.432034,-0.658158,0.274289,-0.340128,1.634267,2.444942e+00
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,-1.752828,-0.115679,0.467130,1.968345,2.488805,-1.249371e+00
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,-1.328180,-0.115679,0.044717,1.371326,0.494884,-1.249371e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.051927,-0.278423,1.229308,0.654903,-0.834397,-2.050756e-16
916,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.901224,0.000000,-0.552169,0.000000,0.000000,-2.050756e-16
917,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.158089,-0.549662,0.219192,-1.494365,-0.834397,-2.050756e-16
918,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.476575,0.000000,1.706817,0.000000,0.000000,-2.050756e-16


In [32]:
X

,age,trestbps,chol,thalch,oldpeak,ca,sex_Female,sex_Male,dataset_Cleveland,dataset_Hungary,...,restecg_normal,restecg_st-t abnormality,exang_False,exang_True,slope_downsloping,slope_flat,slope_upsloping,thal_fixed defect,thal_normal,thal_reversable defect
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.007386,0.698041,0.311021,0.495698,1.349421,-1.249371e+00
1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.432034,1.511761,0.797713,-1.175955,0.589832,4.292099e+00
2,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.432034,-0.658158,0.274289,-0.340128,1.634267,2.444942e+00
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,-1.752828,-0.115679,0.467130,1.968345,2.488805,-1.249371e+00
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,-1.328180,-0.115679,0.044717,1.371326,0.494884,-1.249371e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.051927,-0.278423,1.229308,0.654903,-0.834397,-2.050756e-16
916,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.901224,0.000000,-0.552169,0.000000,0.000000,-2.050756e-16
917,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.158089,-0.549662,0.219192,-1.494365,-0.834397,-2.050756e-16
918,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.476575,0.000000,1.706817,0.000000,0.000000,-2.050756e-16


In [33]:
y = data2[['num']]
y

,num
0,0
1,2
2,1
3,0
4,0
...,...
915,1
916,0
917,2
918,0


In [34]:
y = y.assign(num_binary=y['num'].apply(lambda x: 0 if x == 0 else 1))
y.head()

,num,num_binary
0,0,0
1,2,1
2,1,1
3,0,0
4,0,0


In [35]:
y = y['num_binary']
y

0      0
1      1
2      1
3      0
4      0
      ..
915    1
916    0
917    1
918    0
919    1
Name: num_binary, Length: 920, dtype: int64

In [36]:
splits_d2 = {
        "20/80": stratified_split_data(X, y, test_size=0.8),
        "50/50": stratified_split_data(X, y, test_size=0.5),
        "80/20": stratified_split_data(X, y, test_size=0.2),
    }

## Random Forest

In [37]:
for partition, (X_train, X_test, y_train, y_test) in splits_d2.items():
        print(f"\n--- Results for {partition} Split ---")
        result = evaluate_classifier(rf, params, X_train, X_test, y_train, y_test)
        print(result)


--- Results for 20/80 Split ---
Best parameters: {'max_depth': 20, 'n_estimators': 100}
Training Accuracy: 1.0000 (Error: 0.0000)
Validation Accuracy: 0.8262 (Error: 0.1738)
Testing Accuracy: 0.8302 (Error: 0.1698)
              precision    recall  f1-score   support

           0       0.83      0.78      0.80       329
           1       0.83      0.87      0.85       407

    accuracy                           0.83       736
   macro avg       0.83      0.82      0.83       736
weighted avg       0.83      0.83      0.83       736

Confusion Matrix:
[[255  74]
 [ 51 356]]
{'train_error': 0.0, 'validation_error': np.float64(0.17380574651859693), 'test_error': 0.16983695652173914, 'best_params': {'max_depth': 20, 'n_estimators': 100}}

--- Results for 50/50 Split ---
Best parameters: {'max_depth': None, 'n_estimators': 50}
Training Accuracy: 1.0000 (Error: 0.0000)
Validation Accuracy: 0.8152 (Error: 0.1848)
Testing Accuracy: 0.8304 (Error: 0.1696)
              precision    recall  

## Decision Tree

In [38]:
for partition, (X_train, X_test, y_train, y_test) in splits_d2.items():
    print(f"\n--- Results for {partition} Split ---")
    result = evaluate_classifier(dt, params_dt, X_train, X_test, y_train, y_test)
    print(result)


--- Results for 20/80 Split ---
Best parameters: {'max_depth': None, 'min_samples_split': 10}
Training Accuracy: 0.9348 (Error: 0.0652)
Validation Accuracy: 0.7664 (Error: 0.2336)
Testing Accuracy: 0.7962 (Error: 0.2038)
              precision    recall  f1-score   support

           0       0.80      0.73      0.76       329
           1       0.80      0.85      0.82       407

    accuracy                           0.80       736
   macro avg       0.80      0.79      0.79       736
weighted avg       0.80      0.80      0.80       736

Confusion Matrix:
[[241  88]
 [ 62 345]]
{'train_error': 0.06521739130434778, 'validation_error': np.float64(0.23356248898290144), 'test_error': 0.20380434782608692, 'best_params': {'max_depth': None, 'min_samples_split': 10}}

--- Results for 50/50 Split ---
Best parameters: {'max_depth': 20, 'min_samples_split': 10}
Training Accuracy: 0.9130 (Error: 0.0870)
Validation Accuracy: 0.7304 (Error: 0.2696)
Testing Accuracy: 0.7500 (Error: 0.2500)
    

## KNN

In [39]:
for partition, (X_train, X_test, y_train, y_test) in splits_d2.items():
        print(f"\n--- Results for {partition} Split ---")
        result = evaluate_classifier(knn, params_knn, X_train, X_test, y_train, y_test)
        print(result)


--- Results for 20/80 Split ---
Best parameters: {'n_neighbors': 5, 'weights': 'uniform'}
Training Accuracy: 0.8804 (Error: 0.1196)
Validation Accuracy: 0.8317 (Error: 0.1683)
Testing Accuracy: 0.8166 (Error: 0.1834)
              precision    recall  f1-score   support

           0       0.82      0.75      0.79       329
           1       0.81      0.87      0.84       407

    accuracy                           0.82       736
   macro avg       0.82      0.81      0.81       736
weighted avg       0.82      0.82      0.82       736

Confusion Matrix:
[[247  82]
 [ 53 354]]
{'train_error': 0.11956521739130432, 'validation_error': np.float64(0.16834126564427987), 'test_error': 0.18342391304347827, 'best_params': {'n_neighbors': 5, 'weights': 'uniform'}}

--- Results for 50/50 Split ---
Best parameters: {'n_neighbors': 10, 'weights': 'distance'}
Training Accuracy: 1.0000 (Error: 0.0000)
Validation Accuracy: 0.8391 (Error: 0.1609)
Testing Accuracy: 0.8326 (Error: 0.1674)
            

## Naive Bayes

In [40]:
for partition, (X_train, X_test, y_train, y_test) in splits_d2.items():
    print(f"\n--- Results for {partition} Split ---")
    result = evaluate_classifier(GaussianNB(), naive_bayes_params, X_train, X_test, y_train, y_test)
    print(result)


--- Results for 20/80 Split ---
Best parameters: {'var_smoothing': 1e-07}
Training Accuracy: 0.7500 (Error: 0.2500)
Validation Accuracy: 0.7173 (Error: 0.2827)
Testing Accuracy: 0.7649 (Error: 0.2351)
              precision    recall  f1-score   support

           0       0.68      0.91      0.78       329
           1       0.90      0.65      0.75       407

    accuracy                           0.76       736
   macro avg       0.79      0.78      0.76       736
weighted avg       0.80      0.76      0.76       736

Confusion Matrix:
[[300  29]
 [144 263]]
{'train_error': 0.25, 'validation_error': np.float64(0.28274281685175395), 'test_error': 0.23505434782608692, 'best_params': {'var_smoothing': 1e-07}}

--- Results for 50/50 Split ---
Best parameters: {'var_smoothing': 1e-09}
Training Accuracy: 0.8109 (Error: 0.1891)
Validation Accuracy: 0.7979 (Error: 0.2021)
Testing Accuracy: 0.8196 (Error: 0.1804)
              precision    recall  f1-score   support

           0       0.7

## SVM

In [41]:
for partition, (X_train, X_test, y_train, y_test) in splits_d2.items():
    print(f"\n--- Results for {partition} Split ---")
    result = evaluate_classifier(svm_pipeline, svm_params, X_train, X_test, y_train, y_test)
    print(result)



--- Results for 20/80 Split ---
Best parameters: {'svm__C': 10, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}
Training Accuracy: 0.9783 (Error: 0.0217)
Validation Accuracy: 0.8099 (Error: 0.1901)
Testing Accuracy: 0.8057 (Error: 0.1943)
              precision    recall  f1-score   support

           0       0.82      0.73      0.77       329
           1       0.80      0.87      0.83       407

    accuracy                           0.81       736
   macro avg       0.81      0.80      0.80       736
weighted avg       0.81      0.81      0.80       736

Confusion Matrix:
[[240  89]
 [ 54 353]]
{'train_error': 0.021739130434782594, 'validation_error': np.float64(0.19011105235325232), 'test_error': 0.1942934782608695, 'best_params': {'svm__C': 10, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}}

--- Results for 50/50 Split ---
Best parameters: {'svm__C': 10, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}
Training Accuracy: 0.9674 (Error: 0.0326)
Validation Accuracy: 0.8109 (Error: 0.1891

## Logistic Regression

In [42]:
for partition, (X_train, X_test, y_train, y_test) in splits_d2.items():
    print(f"\n--- Results for {partition} Split ---")
    result = evaluate_classifier(LogisticRegression(), logistic_reg_params, X_train, X_test, y_train, y_test)
    print(result)


--- Results for 20/80 Split ---
Best parameters: {'C': 1, 'max_iter': 7000}
Training Accuracy: 0.8587 (Error: 0.1413)
Validation Accuracy: 0.8206 (Error: 0.1794)
Testing Accuracy: 0.8370 (Error: 0.1630)
              precision    recall  f1-score   support

           0       0.85      0.77      0.81       329
           1       0.83      0.89      0.86       407

    accuracy                           0.84       736
   macro avg       0.84      0.83      0.83       736
weighted avg       0.84      0.84      0.84       736

Confusion Matrix:
[[253  76]
 [ 44 363]]
{'train_error': 0.14130434782608692, 'validation_error': np.float64(0.17935836418120932), 'test_error': 0.1630434782608695, 'best_params': {'C': 1, 'max_iter': 7000}}

--- Results for 50/50 Split ---
Best parameters: {'C': 1, 'max_iter': 7000}
Training Accuracy: 0.8457 (Error: 0.1543)
Validation Accuracy: 0.8152 (Error: 0.1848)
Testing Accuracy: 0.8174 (Error: 0.1826)
              precision    recall  f1-score   support

  

# Dataset 3

In [43]:
data3 = pd.read_csv('adult.csv')
data3.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [44]:
X = data3.drop(columns = ['income'])
X.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States


In [45]:
y = data3[['income']]

In [46]:
y = y.assign(income_binary=y['income'].apply(lambda x: 1 if x == '>50K' else 0))
y.head()

,income,income_binary
0,<=50K,0
1,<=50K,0
2,>50K,1
3,>50K,1
4,<=50K,0


In [47]:
y = np.array(y['income_binary'])
y

array([0, 0, 1, ..., 0, 0, 1])

In [48]:
len(y[y==1])

11687

In [49]:
X.head()#.columns

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States


In [50]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

categorical_columns = [
    'workclass', 'education', 'marital-status', 
    'occupation', 'relationship', 'race', 'native-country'
]

numerical_columns = ['age','fnlwgt', 'educational-num',
       'capital-gain', 'capital-loss', 'hours-per-week']

# Create the preprocessing transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_columns),
         ('num', StandardScaler(), numerical_columns)
    ])

def preprocess_data(X):
    # Fit and transform the entire dataset
    X_transformed = preprocessor.fit_transform(X)

    # Get feature names for numerical and categorical columns
    num_feature_names = numerical_columns
    cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_columns)

    # Combine feature names
    feature_names = list(num_feature_names) + list(cat_feature_names)

    # Create a DataFrame from the transformed array
    X_transformed_df = pd.DataFrame(
        X_transformed, 
        columns=feature_names, 
        index=X.index  # Preserve the original index
    )

    return X_transformed_df

# Optional: If you want to keep numerical columns as well
def preprocess_data_full(X):
    # Fit and transform the data
    X_transformed = preprocessor.fit_transform(X)

    # Get feature names for numerical and categorical columns
    num_feature_names = numerical_columns
    cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_columns)

    # Combine feature names
    feature_names = list(num_feature_names) + list(cat_feature_names)

    # Create a DataFrame from the transformed array
    X_transformed_df = pd.DataFrame(
        X_transformed, 
        columns=feature_names, 
        index=X.index  # Preserve the original index
    )

    return X_transformed_df

In [51]:
X = preprocess_data_full(X)

In [52]:
X.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-0.995129,0.351675,-1.197259,-0.144804,-0.217127,-0.034087
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-0.046942,-0.945524,-0.419335,-0.144804,-0.217127,0.772930
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-0.776316,1.394723,0.747550,-0.144804,-0.217127,-0.034087
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.390683,-0.277844,-0.030373,0.886874,-0.217127,-0.034087
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-1.505691,-0.815954,-0.030373,-0.144804,-0.217127,-0.841104


In [53]:
splits_d3 = {
        "20/80": stratified_split_data(X, y, test_size=0.8),
        "50/50": stratified_split_data(X, y, test_size=0.5),
        "80/20": stratified_split_data(X, y, test_size=0.2),
    }

## Random Forest

In [54]:
for partition, (X_train, X_test, y_train, y_test) in splits_d3.items():
        print(f"\n--- Results for {partition} Split ---")
        result = evaluate_classifier(rf, params, X_train, X_test, y_train, y_test)
        print(result)


--- Results for 20/80 Split ---
Best parameters: {'max_depth': 20, 'n_estimators': 100}
Training Accuracy: 0.9490 (Error: 0.0510)
Validation Accuracy: 0.8532 (Error: 0.1468)
Testing Accuracy: 0.8624 (Error: 0.1376)
              precision    recall  f1-score   support

           0       0.88      0.94      0.91     29724
           1       0.77      0.60      0.68      9350

    accuracy                           0.86     39074
   macro avg       0.83      0.77      0.79     39074
weighted avg       0.86      0.86      0.86     39074

Confusion Matrix:
[[28084  1640]
 [ 3738  5612]]
{'train_error': 0.050982800982800947, 'validation_error': np.float64(0.14680589680589673), 'test_error': 0.13763627987920357, 'best_params': {'max_depth': 20, 'n_estimators': 100}}

--- Results for 50/50 Split ---
Best parameters: {'max_depth': 20, 'n_estimators': 200}
Training Accuracy: 0.9212 (Error: 0.0788)
Validation Accuracy: 0.8615 (Error: 0.1385)
Testing Accuracy: 0.8631 (Error: 0.1369)
           

## Decision Tree

In [55]:
for partition, (X_train, X_test, y_train, y_test) in splits_d3.items():
    print(f"\n--- Results for {partition} Split ---")
    result = evaluate_classifier(dt, params_dt, X_train, X_test, y_train, y_test)
    print(result)


--- Results for 20/80 Split ---
Best parameters: {'max_depth': 10, 'min_samples_split': 5}
Training Accuracy: 0.8744 (Error: 0.1256)
Validation Accuracy: 0.8342 (Error: 0.1658)
Testing Accuracy: 0.8443 (Error: 0.1557)
              precision    recall  f1-score   support

           0       0.90      0.90      0.90     29724
           1       0.68      0.67      0.67      9350

    accuracy                           0.84     39074
   macro avg       0.79      0.78      0.79     39074
weighted avg       0.84      0.84      0.84     39074

Confusion Matrix:
[[26716  3008]
 [ 3076  6274]]
{'train_error': 0.12561425061425058, 'validation_error': np.float64(0.16584766584766586), 'test_error': 0.15570456057736604, 'best_params': {'max_depth': 10, 'min_samples_split': 5}}

--- Results for 50/50 Split ---
Best parameters: {'max_depth': 10, 'min_samples_split': 5}
Training Accuracy: 0.8704 (Error: 0.1296)
Validation Accuracy: 0.8553 (Error: 0.1447)
Testing Accuracy: 0.8573 (Error: 0.1427)
   

## KNN

In [56]:
for partition, (X_train, X_test, y_train, y_test) in splits_d3.items():
        print(f"\n--- Results for {partition} Split ---")
        result = evaluate_classifier(knn, params_knn, X_train, X_test, y_train, y_test)
        print(result)


--- Results for 20/80 Split ---
Best parameters: {'n_neighbors': 10, 'weights': 'uniform'}
Training Accuracy: 0.8580 (Error: 0.1420)
Validation Accuracy: 0.8276 (Error: 0.1724)
Testing Accuracy: 0.8346 (Error: 0.1654)
              precision    recall  f1-score   support

           0       0.86      0.93      0.90     29724
           1       0.70      0.53      0.61      9350

    accuracy                           0.83     39074
   macro avg       0.78      0.73      0.75     39074
weighted avg       0.83      0.83      0.83     39074

Confusion Matrix:
[[27615  2109]
 [ 4354  4996]]
{'train_error': 0.14199426699426698, 'validation_error': np.float64(0.17239967239967235), 'test_error': 0.16540410503147873, 'best_params': {'n_neighbors': 10, 'weights': 'uniform'}}

--- Results for 50/50 Split ---
Best parameters: {'n_neighbors': 10, 'weights': 'uniform'}
Training Accuracy: 0.8645 (Error: 0.1355)
Validation Accuracy: 0.8374 (Error: 0.1626)
Testing Accuracy: 0.8385 (Error: 0.1615)
   

## Naive Bayes

In [57]:
for partition, (X_train, X_test, y_train, y_test) in splits_d3.items():
    print(f"\n--- Results for {partition} Split ---")
    result = evaluate_classifier(GaussianNB(), naive_bayes_params, X_train, X_test, y_train, y_test)
    print(result)


--- Results for 20/80 Split ---
Best parameters: {'var_smoothing': 1e-07}
Training Accuracy: 0.5218 (Error: 0.4782)
Validation Accuracy: 0.4600 (Error: 0.5400)
Testing Accuracy: 0.5091 (Error: 0.4909)
              precision    recall  f1-score   support

           0       0.96      0.37      0.53     29724
           1       0.32      0.96      0.48      9350

    accuracy                           0.51     39074
   macro avg       0.64      0.66      0.51     39074
weighted avg       0.81      0.51      0.52     39074

Confusion Matrix:
[[10940 18784]
 [  398  8952]]
{'train_error': 0.47819410319410316, 'validation_error': np.float64(0.540028665028665), 'test_error': 0.49091467471976247, 'best_params': {'var_smoothing': 1e-07}}

--- Results for 50/50 Split ---
Best parameters: {'var_smoothing': 1e-07}
Training Accuracy: 0.6324 (Error: 0.3676)
Validation Accuracy: 0.5848 (Error: 0.4152)
Testing Accuracy: 0.6310 (Error: 0.3690)
              precision    recall  f1-score   support

 

## SVM

In [60]:
# Randomly sample 20% of the data
X_sampled = X.sample(frac=0.2, random_state=42)
y_sampled = y[X_sampled.index]
from sklearn.model_selection import train_test_split

# Stratified sampling
X_sampled, _, y_sampled, _ = train_test_split(
    X, y, test_size=0.8, stratify=y, random_state=42
)
# Apply sampling before creating splits
X_sampled = X.sample(frac=0.2, random_state=42)
y_sampled = y[X_sampled.index]

# Create new splits with the sampled data
splits_d3_sampled = {
    "20/80": train_test_split(X_sampled, y_sampled, test_size=0.8, random_state=42),
    "50/50": train_test_split(X_sampled, y_sampled, test_size=0.5, random_state=42),
    "80/20": train_test_split(X_sampled, y_sampled, test_size=0.2, random_state=42),
}
for partition, (X_train, X_test, y_train, y_test) in splits_d3_sampled.items():
    print(f"\n--- Results for {partition} Split ---")
    result = evaluate_classifier(svm_pipeline, svm_params, X_train, X_test, y_train, y_test)
    print(result)



--- Results for 20/80 Split ---
Best parameters: {'svm__C': 1, 'svm__gamma': 'scale', 'svm__kernel': 'linear'}
Training Accuracy: 0.8699 (Error: 0.1301)
Validation Accuracy: 0.8520 (Error: 0.1480)
Testing Accuracy: 0.8450 (Error: 0.1550)
              precision    recall  f1-score   support

           0       0.87      0.93      0.90      5956
           1       0.72      0.57      0.63      1859

    accuracy                           0.85      7815
   macro avg       0.80      0.75      0.77      7815
weighted avg       0.84      0.85      0.84      7815

Confusion Matrix:
[[5552  404]
 [ 807 1052]]
{'train_error': 0.13005632360471076, 'validation_error': np.float64(0.14797747055811572), 'test_error': 0.15495841330774152, 'best_params': {'svm__C': 1, 'svm__gamma': 'scale', 'svm__kernel': 'linear'}}

--- Results for 50/50 Split ---
Best parameters: {'svm__C': 10, 'svm__gamma': 'scale', 'svm__kernel': 'linear'}
Training Accuracy: 0.8620 (Error: 0.1380)
Validation Accuracy: 0.8534 (Er

## Logistic Regression

In [61]:
for partition, (X_train, X_test, y_train, y_test) in splits_d3.items():
    print(f"\n--- Results for {partition} Split ---")
    result = evaluate_classifier(LogisticRegression(), logistic_reg_params, X_train, X_test, y_train, y_test)
    print(result)


--- Results for 20/80 Split ---
Best parameters: {'C': 0.1, 'max_iter': 7000}
Training Accuracy: 0.8448 (Error: 0.1552)
Validation Accuracy: 0.8433 (Error: 0.1567)
Testing Accuracy: 0.8518 (Error: 0.1482)
              precision    recall  f1-score   support

           0       0.88      0.94      0.91     29724
           1       0.74      0.58      0.65      9350

    accuracy                           0.85     39074
   macro avg       0.81      0.76      0.78     39074
weighted avg       0.84      0.85      0.85     39074

Confusion Matrix:
[[27849  1875]
 [ 3917  5433]]
{'train_error': 0.15520065520065518, 'validation_error': np.float64(0.15673628173628185), 'test_error': 0.14823156062855092, 'best_params': {'C': 0.1, 'max_iter': 7000}}

--- Results for 50/50 Split ---
Best parameters: {'C': 1, 'max_iter': 7000}
Training Accuracy: 0.8528 (Error: 0.1472)
Validation Accuracy: 0.8512 (Error: 0.1488)
Testing Accuracy: 0.8540 (Error: 0.1460)
              precision    recall  f1-score 

In [ ]:
raise 

In [ ]:
for partition, (X_train, X_test, y_train, y_test) in splits_d3.items():
    print(f"\n--- Results for {partition} Split ---")
    result = evaluate_classifier(svm_pipeline, svm_params, X_train, X_test, y_train, y_test)
    print(result)

# Data 3